In [2]:
import requests
import json
import pymongo

In [48]:
#Create connection to Mongo Database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# create mongo database
db = client.housing_data_db

# Declare the collection
record = db.housing_data_db

In [46]:
#Generate a list of OR county fips

#Generate a list of odd numbers 1-71
odd_numbers = list(range(1,73, 2))

county_fips = []

#Create leading zeros in the county_fips list
for x in range(5):
    county_fips.append("00" + str(odd_numbers[x]))

for x in range(5, len(odd_numbers)):
    county_fips.append("0" + str(odd_numbers[x]))

### Get County Data

##### Census Codes
* NAME = County Name
* B01003_001E = County Population
* B07013_002E = Homeowner Estimate
* B07013_003E = Renter Estimate
* B25064_001E = Median Rent
* B19013_001E = Median Income
* B25063_001E = New Housing Stock (built 2014 or later)

In [49]:
base_url = "https://api.census.gov/data/"

for county in county_fips:
    #2019 County Data
    county_data19 = requests.get(f"{base_url}2019/acs/acs5?get=NAME,B01003_001E,B07013_002E,B07013_003E,B25064_001E,B19013_001E,B25063_001E&for=county:{county}&in=state:41").json()

    #print(json.dumps(county_data19, indent=4, sort_keys=True))

    #Grab county name and take only the name
    full_countyname = county_data19[1][0]

    full_countyname = full_countyname.split()

    county_name = full_countyname[0]

    #Grab all other 2019 variables
    county_pop = county_data19[1][1]
    homeowners = county_data19[1][2]
    renters = county_data19[1][3]
    median_county_rent19 = county_data19[1][4]
    median_income = county_data19[1][5]
    new_housing = county_data19[1][6]
    state_fips = county_data19[1][7]
    county_fips = county_data19[1][8]

    #2015-2018 County Data (median rent only)
    county_data18 = requests.get(f"{base_url}2018/acs/acs5?get=NAME,B25064_001E&for=county:{county}&in=state:41").json()

    #print(json.dumps(county_data18, indent=4, sort_keys=True))

    median_county_rent18 = county_data18[1][1]

    county_data17 = requests.get(f"{base_url}2017/acs/acs5?get=NAME,B25064_001E&for=county:{county}&in=state:41").json()

    median_county_rent17 = county_data17[1][1]

    county_data16 = requests.get(f"{base_url}2016/acs/acs5?get=NAME,B25064_001E&for=county:{county}&in=state:41").json()

    median_county_rent16 = county_data16[1][1]

    county_data15 = requests.get(f"{base_url}2015/acs/acs5?get=NAME,B25064_001E&for=county:{county}&in=state:41").json()

    median_county_rent15 = county_data15[1][1]
    
    county = {
        "county": county_name,
        "pop": county_pop,
        "homeowners": homeowners,
        "renters": renters,
        "county_median_income": median_income,
        "county_median_rent_2019": median_county_rent19,
        "county_median_rent_2018": median_county_rent18,
        "county_median_rent_2017": median_county_rent17,
        "county_median_rent_2016": median_county_rent16,
        "county_median_rent_2015": median_county_rent15,
        "new_housing": new_housing,
        "state_fips": state_fips,
        "county_fips": county_fips
    }
    
    record.insert_one(county)

In [50]:
# Verify results:
results = record.find()
for result in results:
    print(result)

{'_id': ObjectId('6080a4af77534c6ab98257cc'), 'county': 'Baker', 'pop': '16019', 'homeowners': '11034', 'renters': '4207', 'county_median_income': '45998', 'county_median_rent_2019': '674', 'county_median_rent_2018': '675', 'county_median_rent_2017': '658', 'county_median_rent_2016': '636', 'county_median_rent_2015': '608', 'new_housing': '2028', 'state_fips': '41', 'county_fips': '001'}
{'_id': ObjectId('6080a4b477534c6ab98257cd'), 'county': 'Benton', 'pop': '91107', 'homeowners': '50837', 'renters': '33888', 'county_median_income': '62077', 'county_median_rent_2019': '1081', 'county_median_rent_2018': '1019', 'county_median_rent_2017': '962', 'county_median_rent_2016': '893', 'county_median_rent_2015': '862', 'new_housing': '15280', 'state_fips': '41', 'county_fips': '003'}
{'_id': ObjectId('6080a4b877534c6ab98257ce'), 'county': 'Clackamas', 'pop': '410463', 'homeowners': '300358', 'renters': '102701', 'county_median_income': '80484', 'county_median_rent_2019': '1295', 'county_median

### Get State/National Rent Data

In [51]:
state_data19 = requests.get(f"{base_url}2019/acs/acs5?get=NAME,B25064_001E&for=state:41").json()

#print(json.dumps(state_data19, indent=4, sort_keys=True))

median_state_rent19 = state_data19[1][1]

state_data18 = requests.get(f"{base_url}2018/acs/acs5?get=NAME,B25064_001E&for=state:41").json()

median_state_rent18 = state_data18[1][1]

state_data17 = requests.get(f"{base_url}2017/acs/acs5?get=NAME,B25064_001E&for=state:41").json()

median_state_rent17 = state_data17[1][1]

state_data16 = requests.get(f"{base_url}2016/acs/acs5?get=NAME,B25064_001E&for=state:41").json()

median_state_rent16 = state_data16[1][1]

state_data15 = requests.get(f"{base_url}2015/acs/acs5?get=NAME,B25064_001E&for=state:41").json()

median_state_rent15 = state_data15[1][1]

state = {
    "state_median_rent_2019": median_state_rent19,
    "state_median_rent_2018": median_state_rent18,
    "state_median_rent_2017": median_state_rent17,
    "state_median_rent_2016": median_state_rent16,
    "state_median_rent_2015": median_state_rent15,
    "data": "state"
}

record.insert_one(state)

# Verify results:
results = record.find()
for result in results:
    print(result)

{'_id': ObjectId('6080a4af77534c6ab98257cc'), 'county': 'Baker', 'pop': '16019', 'homeowners': '11034', 'renters': '4207', 'county_median_income': '45998', 'county_median_rent_2019': '674', 'county_median_rent_2018': '675', 'county_median_rent_2017': '658', 'county_median_rent_2016': '636', 'county_median_rent_2015': '608', 'new_housing': '2028', 'state_fips': '41', 'county_fips': '001'}
{'_id': ObjectId('6080a4b477534c6ab98257cd'), 'county': 'Benton', 'pop': '91107', 'homeowners': '50837', 'renters': '33888', 'county_median_income': '62077', 'county_median_rent_2019': '1081', 'county_median_rent_2018': '1019', 'county_median_rent_2017': '962', 'county_median_rent_2016': '893', 'county_median_rent_2015': '862', 'new_housing': '15280', 'state_fips': '41', 'county_fips': '003'}
{'_id': ObjectId('6080a4b877534c6ab98257ce'), 'county': 'Clackamas', 'pop': '410463', 'homeowners': '300358', 'renters': '102701', 'county_median_income': '80484', 'county_median_rent_2019': '1295', 'county_median

In [52]:
us_data19 = requests.get(f"{base_url}2019/acs/acs5?get=NAME,B25064_001E&for=us:*").json()

#print(json.dumps(us_data19, indent=4, sort_keys=True))

median_us_rent19 = us_data19[1][1]

us_data18 = requests.get(f"{base_url}2018/acs/acs5?get=NAME,B25064_001E&for=us:*").json()

median_us_rent18 = us_data18[1][1]

us_data17 = requests.get(f"{base_url}2017/acs/acs5?get=NAME,B25064_001E&for=us:*").json()

median_us_rent17 = us_data17[1][1]

us_data16 = requests.get(f"{base_url}2016/acs/acs5?get=NAME,B25064_001E&for=us:*").json()

median_us_rent16 = us_data16[1][1]

us_data15 = requests.get(f"{base_url}2015/acs/acs5?get=NAME,B25064_001E&for=us:*").json()

median_us_rent15 = us_data15[1][1]

national = {
    "us_median_rent_2019": median_us_rent19,
    "us_median_rent_2018": median_us_rent18,
    "us_median_rent_2017": median_us_rent17,
    "us_median_rent_2016": median_us_rent16,
    "us_median_rent_2015": median_us_rent15,
    "data": "national"
}

record.insert_one(national)

# Verify results:
results = record.find()
for result in results:
    print(result)

{'_id': ObjectId('6080a4af77534c6ab98257cc'), 'county': 'Baker', 'pop': '16019', 'homeowners': '11034', 'renters': '4207', 'county_median_income': '45998', 'county_median_rent_2019': '674', 'county_median_rent_2018': '675', 'county_median_rent_2017': '658', 'county_median_rent_2016': '636', 'county_median_rent_2015': '608', 'new_housing': '2028', 'state_fips': '41', 'county_fips': '001'}
{'_id': ObjectId('6080a4b477534c6ab98257cd'), 'county': 'Benton', 'pop': '91107', 'homeowners': '50837', 'renters': '33888', 'county_median_income': '62077', 'county_median_rent_2019': '1081', 'county_median_rent_2018': '1019', 'county_median_rent_2017': '962', 'county_median_rent_2016': '893', 'county_median_rent_2015': '862', 'new_housing': '15280', 'state_fips': '41', 'county_fips': '003'}
{'_id': ObjectId('6080a4b877534c6ab98257ce'), 'county': 'Clackamas', 'pop': '410463', 'homeowners': '300358', 'renters': '102701', 'county_median_income': '80484', 'county_median_rent_2019': '1295', 'county_median